# Imports

In [1]:
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

---

# Carga de datos

In [14]:
features = pd.read_pickle("../../features/auctions_train_ventana_1.pkl")
targets = pd.read_pickle("../../targets/targets_ventana_1_auctions.pkl")
train = features.merge(targets, left_index=True, right_index=True)

---

# Preparación del set de datos para XGBoost

In [64]:
X, y = train.iloc[:,:-1], train.iloc[:,-1]
data_dmatrix = xgb.DMatrix(data=X,label=y)
X_train, X_test, y_train, y_test = \
    train_test_split(X, y, test_size=0.2, random_state=123)

In [65]:
xg_reg = xgb.XGBRegressor(colsample_bytree = 0.7, learning_rate = 1,
                n_estimators = 100, n_jobs=6, max_depth=7, objective="reg:linear")

In [66]:
xg_reg.fit(X_train,y_train)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=0.7, gamma=0, learning_rate=1, max_delta_step=0,
       max_depth=7, min_child_weight=1, missing=None, n_estimators=100,
       n_jobs=6, nthread=None, objective='reg:linear', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1)

In [67]:
preds = xg_reg.predict(X_test)
rmse = np.sqrt(mean_squared_error(y_test, preds))
print("RMSE: %f" % (rmse))

RMSE: 93545.206149


---

# Pruebas

In [19]:
features1 = pd.read_pickle("../../features/auctions_train_ventana_1.pkl")
targets1 = pd.read_pickle("../../targets/targets_ventana_1_auctions.pkl")
train = features1.merge(targets1, left_index=True, right_index=True)

features2 = pd.read_pickle("../../features/auctions_train_ventana_2.pkl")
targets2 = pd.read_pickle("../../targets/targets_ventana_2_auctions.pkl")
train.append(features2.merge(targets2, left_index=True, right_index=True))

X, y = train.iloc[:,:-1], train.iloc[:,-1]
X_train, X_test, y_train, y_test = \
    train_test_split(X, y, test_size=0.2, random_state=123)

dtrain = xgb.DMatrix(data=X_train, label=y_train)
dtest = xgb.DMatrix(data=X_test, label=y_test)

param = {'max_depth':5,
         'eta':0.1,
         'silent':1,
         'objective':'reg:linear',
         'eval_metric' : 'rmse',
         'obj' : 'reg:squarederror',
         'colsample_bytree' : 0.7,
         'min_child_weight' : 0.5,
         'random_state' : 0,
         'reg_alpha' : 0,
         'reg_lambda' : 1,
         'scale_pos_weight' : 1,
         'learning_rate' : 0.01}


num_round = 5000
stopping = 20
bst = xgb.train(param, dtrain, num_round, evals=[(dtrain, "train"), (dtest, "test")], early_stopping_rounds=stopping)
# make prediction
#preds = bst.predict(dtest)

[0]	train-rmse:107901	test-rmse:104304
Multiple eval metrics have been passed: 'test-rmse' will be used for early stopping.

Will train until test-rmse hasn't improved in 20 rounds.
[1]	train-rmse:107311	test-rmse:103738
[2]	train-rmse:106739	test-rmse:103183
[3]	train-rmse:106166	test-rmse:102633
[4]	train-rmse:105644	test-rmse:102130
[5]	train-rmse:105137	test-rmse:101636
[6]	train-rmse:104594	test-rmse:101111
[7]	train-rmse:104050	test-rmse:100588
[8]	train-rmse:103576	test-rmse:100133
[9]	train-rmse:103047	test-rmse:99628.3
[10]	train-rmse:102571	test-rmse:99168
[11]	train-rmse:102060	test-rmse:98682.3
[12]	train-rmse:101557	test-rmse:98198.6
[13]	train-rmse:101057	test-rmse:97721.8
[14]	train-rmse:100568	test-rmse:97256.4
[15]	train-rmse:100082	test-rmse:96795.3
[16]	train-rmse:99606.4	test-rmse:96341.6
[17]	train-rmse:99175.1	test-rmse:95930
[18]	train-rmse:98746.6	test-rmse:95519.2
[19]	train-rmse:98295.6	test-rmse:95087.9
[20]	train-rmse:97847.1	test-rmse:94656.9
[21]	train-rms

---

In [72]:
import lightgbm as lgb
lgb_model = lgb.LGBMRegressor(metric='rmse', n_estimators=50, colsample_bytree=0.8)
#lgb_model = lgb.LGBMRegressor(boosting_type='dart', num_leaves=7, n_estimators=1000, metric='rmse',
#                               learning_rate=0.05, colsample_bytree=0.8)
lgb_model.fit(X_train, y_train)
preds = lgb_model.predict(X_test)
rmse = np.sqrt(mean_squared_error(y_test, preds))
print("RMSE: %f" % (rmse))

RMSE: 70891.701428


In [59]:
preds_xgb = bst.predict(dtest)
preds_lgbm = lgb_model.predict(X_test)

In [60]:
p1 = 0.45
p2 = 0.55

preds = p1*preds_xgb + p2*preds_lgbm
rmse = np.sqrt(mean_squared_error(y_test, preds))
print("RMSE: %f" % (rmse))

RMSE: 70581.867120
